In [1]:
import random
import time
import math
import csv
import os
import chessAI
from pathlib import Path
from time import monotonic
from multiprocessing import Process, Queue
from chessEngine import GameState
from chessAI import get_best_move, run_ai_loop
from stockfishHandler import StockfishPlayer

/Users/arjunvijayanup/Documents/3rd Trimester/Project in Maths Modelling/Chess/project-chess-final/project-ai-chess-engine/chess_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Deterministic check for move search

In [2]:
def best_move_once():
    gs = GameState()
    legal = gs.get_valid_moves()
    random.seed(12345)  # stabilize any random tie-breakers
    mv = get_best_move(gs, legal)
    try:
        return mv.get_chess_notation()
    except Exception:
        return str(mv)

In [3]:
a = best_move_once()
b = best_move_once()
print("Run1:", a)
print("Run2:", b)
print("DETERMINISTIC" if a == b else "NON-DETERMINISTIC")

Run1: e2e4
Run2: e2e4
DETERMINISTIC


# Move generation Correctness & Speed

In [4]:
def perft(gs, depth):
    if depth == 0:
        return 1
    nodes = 0
    for m in gs.get_valid_moves():
        gs.make_move(m)
        nodes += perft(gs, depth - 1)
        gs.undo_move()
    return nodes

In [5]:
def divide(gs, depth):
    total = 0
    for m in gs.get_valid_moves():
        gs.make_move(m)
        c = perft(gs, depth - 1)
        gs.undo_move()
        name = getattr(m, "get_chess_notation", lambda: str(m))()
        print(f"{name:<8} {c}")
        total += c
    print(f"Total: {total}")
    return total

In [6]:
gs = GameState()
tests = [(1, 20), (2, 400), (3, 8902), (4, 197281)]
for d, expected in tests:
    t0 = time.time()
    n = perft(gs, d)
    dt = time.time() - t0
    status = "PASS" if n == expected else f"FAIL (expected {expected})"
    print(f"depth {d}: nodes={n}  {status}  time={dt:.3f}s  nps={int(n/max(dt,1e-6))}")

depth 1: nodes=20  PASS  time=0.002s  nps=11305
depth 2: nodes=400  PASS  time=0.012s  nps=34140
depth 3: nodes=8902  PASS  time=0.077s  nps=115421
depth 4: nodes=197281  PASS  time=1.663s  nps=118627


# Stockfish vs Our AI engine (ELO & Match Stats)

-   N = wins + draws + losses.
-   Score S = (wins + 0.5*draws) / N.
-   Rating difference $\Delta$ vs that Stockfish config (depth/time) using the Elo logistic model:

$$
\Delta = -400 \log_{10}\!\left(\frac{1}{S} - 1\right)
$$

In [22]:
NUM_GAMES = 1
OUR_DEPTH = 5
STOCKFISH_DEPTH = 3
MAX_PLIES = 300
AI_TIMEOUT_SEC = 90
PRINT_PER_MOVE = False

chessAI.MAX_DEPTH = OUR_DEPTH

LOG_TO_CSV = True
CSV_PATH = f"arena_results_sf{STOCKFISH_DEPTH}_ai{OUR_DEPTH}.csv"

In [8]:
def _ensure_csv_header(path):
    need_header = not os.path.exists(path) or os.path.getsize(path) == 0
    if need_header:
        with open(path, "w", newline="") as f:
            w = csv.writer(f)
            w.writerow([
                "game_index", "ai_color", "result",
                "plies", "sf_depth", "ai_depth",
                "ai_nodes", "ai_time_ms", "ai_nps",
                "tt_probes", "tt_hits", "tt_hit_rate",
                "tt_stores", "beta_cutoffs", "first_move_cutoffs",
                "avg_branch", "killer_uses", "history_uses",
                "ai_moves", "sf_time_ms", "game_wall_ms"
            ])

def _append_csv_row(path, row):
    with open(path, "a", newline="") as f:
        w = csv.writer(f)
        w.writerow(row)

def _log_game_csv(path, game_idx, result, ai_is_white, agg):
    row = [
        game_idx + 1,
        "white" if ai_is_white else "black",
        result,
        OUR_DEPTH,
        STOCKFISH_DEPTH,
        agg["plies"],
        agg["ai_moves"],
        f"{agg['ai_time_ms']:.1f}",
        f"{(agg['ai_time_ms']/max(1, agg['ai_moves'])):.1f}",
    ]
    with open(path, "a", newline="") as f:
        csv.writer(f).writerow(row)

In [9]:
def _empty_agg():
    return {
        "nodes": 0,
        "ms": 0.0,
        "sf_ms": 0.0,
        "tt_probes": 0,
        "tt_hits": 0,
        "tt_stores": 0,
        "beta_cutoffs": 0,
        "first_move_cutoffs": 0,
        "killer_uses": 0,
        "history_uses": 0,
        "branch_sum": 0,
        "branch_samples": 0,
        "plies": 0,   # total half-moves in the game
        "ai_moves": 0 # number of AI root searches performed
    }


def _accumulate(agg, stats):
    """Add a single SearchStats into the aggregate dict."""
    agg["nodes"] += getattr(stats, "nodes")
    agg["ms"] += getattr(stats, "ms")
    agg["tt_probes"] += getattr(stats, "tt_probes")
    agg["tt_hits"] += getattr(stats, "tt_hits")
    agg["tt_stores"] += getattr(stats, "tt_stores")
    agg["beta_cutoffs"] += getattr(stats, "beta_cutoffs")
    agg["first_move_cutoffs"] += getattr(stats, "first_move_cutoffs")
    agg["killer_uses"] += getattr(stats, "killer_uses")
    agg["history_uses"] += getattr(stats, "history_uses")
    agg["branch_sum"] += getattr(stats, "branch_sum")
    agg["branch_samples"] += getattr(stats, "branch_samples")
    agg["ai_moves"] += 1


def _format_agg(agg):
    """Human-readable one-liner for game stats."""
    nodes = agg["nodes"]
    ms = agg["ms"]
    sf_ms = agg.get("sf_ms", 0.0)
    nps = (nodes / (ms / 1000.0)) if ms > 1e-9 else 0.0
    ttp = agg["tt_probes"]
    tth = agg["tt_hits"]
    tt_hit_rate = (100.0 * tth / ttp) if ttp else 0.0
    avg_branch = (agg["branch_sum"] / agg["branch_samples"]) if agg["branch_samples"] else 0.0
    return (
        f"nodes={nodes:,}, time={ms:.1f} ms, NPS={nps:,.0f}, "
        f"TT hit={tt_hit_rate:.1f}% ({tth}/{ttp}), stores={agg['tt_stores']}, "
        f"beta-cutoffs={agg['beta_cutoffs']}, first-move-cutoffs={agg['first_move_cutoffs']}, "
        f"avg-branch={avg_branch:.2f}, killer-uses={agg['killer_uses']}, "
        f"history-uses={agg['history_uses']}, AI-moves={agg['ai_moves']}, plies={agg['plies']}, "
        f"SFtime={sf_ms:.1f} ms"
    )


In [ ]:
def start_ai_worker(depth):
    iq, oq = Queue(), Queue()
    print(f"[child] chessAI.MAX_DEPTH set to {depth}")
    p = Process(target=chessAI.run_ai_loop, args=(iq, oq, depth), daemon=True)
    p.start()
    return p, iq, oq


def stop_ai_worker(proc):
    if proc and proc.is_alive():
        proc.terminate()
        proc.join(timeout=2)

In [11]:
def get_ai_move(worker_in, worker_out, gs, legal_moves):
    pos_key = len(gs.moves_log)
    worker_in.put((pos_key, gs, legal_moves))
    deadline = monotonic() + AI_TIMEOUT_SEC
    while monotonic() < deadline:
        if not worker_out.empty():
            pos_key_ret, mv, stats = worker_out.get()
            if pos_key_ret == len(gs.moves_log):
                return mv, stats
    return None, None # timeout

In [12]:
def play_one_game(sf, worker_in, worker_out, ai_is_white=True):
    gs = GameState()
    legal = gs.get_valid_moves()
    plies = 0
    agg = _empty_agg()

    while True:
        # terminal?
        if not legal: # no legal moves left
            if gs.is_stalemate:
                agg["plies"] = plies
                return "draw", agg
            winner_is_white = not gs.white_to_move  # side NOT to move was just checkmated
            agg["plies"] = plies
            if winner_is_white == ai_is_white:
                return "ai", agg
            else:
                return "stockfish", agg 
            #return "ai", agg if winner_is_white == ai_is_white else "stockfish", agg

        if getattr(gs, "is_threefold_repetition", False) or getattr(gs, "is_fifty_move_draw", False) or getattr(gs, "is_insufficient_material", False):
            agg["plies"] = plies
            return "draw", agg
        
        if plies >= MAX_PLIES:
            agg["plies"] = plies
            return "draw", agg

        ai_turn = (gs.white_to_move and ai_is_white) or ((not gs.white_to_move) and (not ai_is_white))

        if ai_turn:
            mv, stats = get_ai_move(worker_in, worker_out, gs, legal)
            print(f"[AI] requested depth = {stats.req_depth}, max ply seen = {stats.max_ply_seen}")
            if stats.max_ply_seen < stats.req_depth:
                print(f"[warn] max_ply_seen {stats.max_ply_seen} < requested {stats.req_depth} (heavy pruning or early terminal?)")
            
            if mv is None:  # timeout / fail-safe → resign
                agg["plies"] = plies
                return "stockfish", agg
            _accumulate(agg, stats)
            if PRINT_PER_MOVE:
                print("AI move statistics:", _format_agg({**agg, "plies": plies}) )
        else:
            mv = sf.get_best_move(gs)
            t_sf0 = monotonic()
            agg["sf_ms"] += (monotonic()-t_sf0) * 1000.0 

        # make move (promotion default 'Q' is fine in your engine)
        gs.make_move(mv)
        legal = gs.get_valid_moves()
        plies += 1


In [13]:
def elo_diff_from_score(score):
    score = min(max(score, 1e-6), 1-1e-6)
    return -400.0 * math.log10(1/score - 1)

In [ ]:
sf = StockfishPlayer(base_dir=os.getcwd(), depth=STOCKFISH_DEPTH)
worker_proc, worker_in, worker_out = start_ai_worker(OUR_DEPTH)

_ensure_csv_header(CSV_PATH)
ai_wins = sf_wins = draws = 0
total_ai_ms = 0.0
total_ai_moves = 0

try:
    for g in range(NUM_GAMES):
        ai_is_white = (g % 2 == 0)
        t0 = monotonic()
        result, agg = play_one_game(sf, worker_in, worker_out, ai_is_white=ai_is_white)
        
        if result == "ai":
            ai_wins += 1
        elif result == "stockfish":
            sf_wins += 1
        else:
            draws += 1
        print(f"Game {g+1}/{NUM_GAMES}: {result} ({'AI as White' if ai_is_white else 'AI as Black'})")
        print("Stats:", _format_agg(agg))

        nodes = agg["nodes"]
        ms = agg["ms"]
        nps = (nodes / (ms / 1000.0)) if ms > 1e-9 else 0.0
        ttp = agg["tt_probes"]
        tth = agg["tt_hits"]
        tt_hit_rate = (100.0 * tth / ttp) if ttp else 0.0
        avg_branch = (agg["branch_sum"] / agg["branch_samples"]) if agg["branch_samples"] else 0.0
        game_wall_ms = (monotonic() - t0) * 1000.0 
        sf_ms = agg.get("sf_ms", 0.0)

        _append_csv_row(CSV_PATH, [
                g + 1,
                "white" if ai_is_white else "black",
                result,
                agg["plies"],
                STOCKFISH_DEPTH,
                OUR_DEPTH,
                nodes,
                f"{ms:.1f}",
                f"{nps:.0f}",
                ttp,
                tth,
                f"{tt_hit_rate:.1f}",
                agg["tt_stores"],
                agg["beta_cutoffs"],
                agg["first_move_cutoffs"],
                f"{avg_branch:.2f}",
                agg["killer_uses"],
                agg["history_uses"],
                agg["ai_moves"],
                f"{sf_ms:.1f}",
                f"{game_wall_ms:.1f}",
            ])
        # print per-game summary (includes time)
        #print(_format_game_line(g, NUM_GAMES, result, ai_is_white, agg))

    total = ai_wins + sf_wins + draws
    print("\n=== Final tally ===")
    print(f"AI wins:        {ai_wins}")
    print(f"Stockfish wins: {sf_wins}")
    print(f"Draws:          {draws}")
    if total:
        score = (ai_wins + 0.5*draws) / total
        print(f"AI score: {100*score:.1f}%")
        print(f"ΔElo vs SF@depth{STOCKFISH_DEPTH}: {elo_diff_from_score(score):+.0f}")

    if LOG_TO_CSV:
        print(f"\nCSV saved to: {os.path.abspath(CSV_PATH)}")
finally:
    stop_ai_worker(worker_proc)
    sf.quit_engine()

[child] chessAI.MAX_DEPTH set to 5
[chessAI] Using MAX_DEPTH=5
[AI] requested depth = 0, max ply seen = 0
[AI] requested depth = 0, max ply seen = 0
[AI] requested depth = 0, max ply seen = 0
[AI] requested depth = 5, max ply seen = 5
[AI] requested depth = 5, max ply seen = 5
[AI] requested depth = 5, max ply seen = 5
[AI] requested depth = 5, max ply seen = 5
[AI] requested depth = 5, max ply seen = 5
[AI] requested depth = 5, max ply seen = 5
[AI] requested depth = 5, max ply seen = 5
[AI] requested depth = 5, max ply seen = 5
[AI] requested depth = 5, max ply seen = 5
